## Kidney Exchange

with integer programming and Gurobi

Source: http://examples.gurobi.com/kidney-exchange/

器官移植问题, 在捐助者和病人之间进行配型. 

In this example we'll solve the Kidney Exchange Problem: how to exchange kidneys between donors and patients in need of a transplant.

<img height=200 width=400 src=http://examples.gurobi.com/kidney-exchange/screenshot.png>


## Problem Description


在美国, 有10万人在等候移植列表. 病人要等候4-5年时间. 4000人在等候时死亡.

According to the New York Times, in the United States there are more than 100,000 people, on the transplant list, waiting for a kidney. Patients typically wait four to five years before a transplant. In 2014, more than 4,000 people died while waiting on the transplant list.

New software and algorithms, many of them based on integer programming, have recently been developed to better match patients with donors. In this example, we will see how an integer programming model based on exchanging kidneys can help more transplants happen.

Often a patient in need of a transplant has a friend or family member who is willing to donate their kidney. However, because of differences in blood types and proteins in the blood, the potential donor may be incompatiable with the patient. Thus a transplant cannot be made, or if the transplant were to occur, the recipient would have a high chance of rejecting the donor's kidney.

In this example we consider four types of patients (donors and recipients) categorized by blood type and proteins. Each patient type is represented by a color: blue, orange, green, or red. The different compatibility factors between the different types are shown in the table below:


最多支持3对. 避免不可控的事情发生. 

However, there are issues with longer cycles. In a longer cycle more people are affected if an exchange fails. In addition, if all transplants are done simultaneously (to avoid donors backing out once their partner has received a kidney), more medical staff and operating rooms are required with longer cycles. Therefore, in this example, we restrict the number of donor/recipient pairs in a cycle to be at most 3.


## Implementation

In [1]:
from gurobipy import *

vertices  = range(5)
edges = { (0,1) : 1, (1,0) : 1, (0,2) : 1, (2,0) : 1,
          (0,4) : 1, (4,0) : 1, (1,4) : 1, (4,1) : 1,
          (1,3) : 1, (3,1) : 1, (2,3) : 1, (3,2) : 1,
          (3,4) : 1, (4,3) : 1 }

def twoCycle(vertices, edges):
    '''
    Returns a dictionary of 2 cycles. Keys: (u,v), Value: weight of cycle
    Note that u < v to not double count cycles.
    '''
    twoCycles = {}
    for edge in edges:
        u = edge[0]
        v = edge[1]
        if (u < v and (v,u) in edges):
            twoCycles[(u,v)] = edges[(u,v)] + edges[(v,u)]
    return twoCycles

def threeCycle(vertices, edges):
    '''
    Returns a dictionary of 3 cycles. Keys: (u,w,v), Value: weight of cycle
    Note that w is always the lowest numbered vertex to not double
    (or triple) count cycles.
    '''
    threeCycles = {}
    for edge in edges:
        u = edge[0]
        v = edge[1]
        for w in vertices:
            if (w >= u or w >= v ):
                break
            if ( (u,w) in edges and (w,v) in edges ):
                threeCycles[(u,w,v)] = edges[(u,v)] + edges[(u,w)] + edges[(w,v)]
    return threeCycles

twoCycles = twoCycle(vertices, edges)
threeCycles = threeCycle(vertices, edges)

m = Model()

c = {}

for cycle in twoCycles:
    c[cycle] = m.addVar(vtype=GRB.BINARY, name="c_%s" % str(cycle))

for cycle in threeCycles:
    c[cycle] = m.addVar(vtype=GRB.BINARY, name="c_%s" % str(cycle))

m.update()

for v in vertices:
  constraint = []
  for cycle in c:
      if (v in cycle):
          constraint.append(c[cycle])
  if constraint:
      m.addConstr( quicksum( constraint[i] for i in range(len(constraint)) ) <= 1 , name="v%d" % v)

m.setObjective( quicksum( c[cycle] * twoCycles[cycle] for cycle in twoCycles ) +
                quicksum( c[cycle] * threeCycles[cycle] for cycle in threeCycles ),
                GRB.MAXIMIZE )

m.optimize()
      

Optimize a model with 5 rows, 11 columns and 26 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolved: 5 rows, 9 columns, 20 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)

Root relaxation: objective 5.000000e+00, 7 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.00000    0    3    4.00000    5.00000  25.0%     -    0s
H    0     0                       5.0000000    5.00000  0.00%     -    0s

Explored 0 nodes (8 simplex iterations) in 0.17 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 5 4 
Pool objective bound 5

Optimal solution 

## Optimal Result

```
生成2对, 或3对的 配型组合
c_(2, 3) 1.000000
c_(4, 0, 1) 1.000000
```

In [2]:
for v in m.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

c_(2, 3) 1.000000
c_(4, 0, 1) 1.000000
